In [ ]:
!pip install datasets rouge-score bert-score transformers

In [ ]:
import torch
from datasets import load_dataset
from transformers import T5ForConditionalGeneration, T5Tokenizer
from rouge_score import rouge_scorer
from bert_score import score

In [ ]:
# 1. Load dataset (1000 samples)
dataset = load_dataset("cnn_dailymail", "3.0.0", split="test[:1000]")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
# 2. Load T5-base model + tokenizer
model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# 3. Generate summaries
references, predictions = [], []
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
for sample in dataset:
    article = sample["article"]
    ref_summary = sample["highlights"]

    # Prepend task prefix
    input_text = "summarize: " + article
    inputs = tokenizer(input_text, truncation=True, padding="longest", max_length=512, return_tensors="pt").to(device)

    # Generate
    summary_ids = model.generate(inputs["input_ids"], max_length=150, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
    pred_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    references.append(ref_summary)
    predictions.append(pred_summary)


In [ ]:
# 4. Evaluation: ROUGE
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [ ]:
rouge1, rouge2, rougel = 0, 0, 0
for ref, pred in zip(references, predictions):
    scores = scorer.score(ref, pred)
    rouge1 += scores['rouge1'].fmeasure
    rouge2 += scores['rouge2'].fmeasure
    rougel += scores['rougeL'].fmeasure

In [ ]:
n = len(predictions)
print("ROUGE-1:", rouge1/n)
print("ROUGE-2:", rouge2/n)
print("ROUGE-L:", rougel/n)

In [ ]:
# 5. Evaluation: BERTScore
P, R, F1 = score(predictions, references, lang="en", verbose=True)
print("BERTScore F1:", F1.mean().item())# 6. Show sample outputs
for i in range(3):
    print("\nARTICLE:", dataset[i]["article"][:400], "...")
    print("REFERENCE SUMMARY:", dataset[i]["highlights"])
    print("T5 SUMMARY:", predictions[i])